In [3]:
import sqlite3
import pandas as pd

In [19]:
#need to find a way so this isn't called every time the page is run, use g global variable
#do some data cleaning: remove profiles that list 

def make_table():
    with sqlite3.connect("voter_data.db") as conn:
        df = pd.read_csv("~/Documents/GitHub/course-project-16b/alaska_presidentialelection.csv")
        df = df[["rank1","rank2","rank3","rank4","rank5"]]
        df.to_sql("votes",conn,index=False,if_exists="replace")
        

In [22]:
with sqlite3.connect("voter_data.db") as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM votes LIMIT 5")
    print(cursor.fetchall())
    

[('Joseph R. Biden', 'Amy Klobuchar', 'Pete Buttigieg', 'Tom Steyer', 'Elizabeth Warren'), ('Joseph R. Biden', 'skipped', 'skipped', 'skipped', 'skipped'), ('Joseph R. Biden', 'skipped', 'skipped', 'skipped', 'skipped'), ('Bernie Sanders', 'Joseph R. Biden', 'skipped', 'skipped', 'skipped'), ('Joseph R. Biden', 'Elizabeth Warren', 'skipped', 'skipped', 'skipped')]


In [59]:
with sqlite3.connect("voter_data.db") as conn:
    cursor = conn.cursor()
    results = []
    for col in ["rank1","rank2","rank3","rank4","rank5"]:
        cmd = f"SELECT DISTINCT {col} FROM votes"
        cursor.execute(cmd)
        results += list(zip(*cursor.fetchall()))[0]
        
results = list(set(results))
results
#clean later to remove overvote, skipped, undeclared

['overvote',
 'skipped',
 'Tulsi Gabbard',
 'Elizabeth Warren',
 'Pete Buttigieg',
 'Michael R. Bloomberg',
 'Tom Steyer',
 'Joseph R. Biden',
 'Undeclared',
 'Bernie Sanders',
 'Amy Klobuchar']

In [73]:
results = ['Tulsi Gabbard',
 'Elizabeth Warren',
 'Pete Buttigieg',
 'Michael R. Bloomberg',
 'Tom Steyer',
 'Joseph R. Biden',
 'Bernie Sanders',
 'Amy Klobuchar']

In [34]:
def plurality():
    with sqlite3.connect("voter_data.db") as conn:
        cursor = conn.cursor()
        cmd = """
        SELECT rank1, COUNT(rank1)
        FROM votes
        GROUP BY rank1
        ORDER BY COUNT(rank1) DESC
        """
        cursor.execute(cmd)
        return cursor.fetchall()

In [35]:
plurality()

[('Joseph R. Biden', 9860),
 ('Bernie Sanders', 7762),
 ('Elizabeth Warren', 1399),
 ('Pete Buttigieg', 247),
 ('Amy Klobuchar', 131),
 ('Tulsi Gabbard', 124),
 ('Michael R. Bloomberg', 97),
 ('Undeclared', 64),
 ('Tom Steyer', 64),
 ('overvote', 32),
 ('skipped', 31)]

In [74]:
def borda(point_dict={1:1,2:2,3:3,4:4,5:5}):
    #ask if there's a more efficient way to do this
    
    count = {}
    with sqlite3.connect("voter_data.db") as conn:
        cursor = conn.cursor()
        for result in results:
            for col in [1,2,3,4,5]:
                cmd = f"SELECT {point_dict[col]}*COUNT(rank{col}) FROM votes WHERE (rank{col}=='{result}')"
                cursor.execute(cmd)
                count[result] = count.get(result,0) + cursor.fetchone()[0]
    return [(i,j) for i,j in count.items()]

In [75]:
borda()

[('Tulsi Gabbard', 6817),
 ('Elizabeth Warren', 23377),
 ('Pete Buttigieg', 19055),
 ('Michael R. Bloomberg', 8535),
 ('Tom Steyer', 7766),
 ('Joseph R. Biden', 29815),
 ('Bernie Sanders', 28724),
 ('Amy Klobuchar', 19112)]

In [83]:
def preference_profiles():
    with sqlite3.connect("voter_data.db") as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT *,COUNT(*) FROM votes \
        GROUP BY rank1,rank2,rank3,rank4,rank5 \
        ORDER BY COUNT(*) DESC")
    return cursor.fetchall()
 
len(preference_profiles())

2467

In [ ]:
def IRV():
    can_win = results
    to_return = 
    with sqlite3.connect("voter_data.db") as conn:
        cursor = conn.cursor()
        while len(can_win)>1:
            cmd = f"SELECT COUNT(rank1)"
            
#figure out a way to implement this

In [92]:
def TopTwo():
    plural = plurality()
    top_two = [plural[0][0],plural[1][0]]
    with sqlite3.connect("voter_data.db") as conn:
        cursor = conn.cursor()
        #hard-coded because I have no idea how else to do this
        cmd = f"""
        SELECT COUNT(*) FROM votes
        WHERE (rank1 == '{top_two[0]}' AND rank2 == '{top_two[1]}')
        OR (rank1 == '{top_two[0]}' AND rank3 == '{top_two[1]}')
        OR (rank1 == '{top_two[0]}' AND rank4 == '{top_two[1]}')
        OR (rank1 == '{top_two[0]}' AND rank5 == '{top_two[1]}')
        OR (rank2 == '{top_two[0]}' AND rank3 == '{top_two[1]}')
        OR (rank2 == '{top_two[0]}' AND rank4 == '{top_two[1]}')
        OR (rank2 == '{top_two[0]}' AND rank5 == '{top_two[1]}')
        OR (rank3 == '{top_two[0]}' AND rank4 == '{top_two[1]}')
        OR (rank3 == '{top_two[0]}' AND rank5 == '{top_two[1]}')
        OR (rank4 == '{top_two[0]}' AND rank5 == '{top_two[1]}')
        OR (rank5 == '{top_two[0]}' AND rank1 != '{top_two[1]}' AND rank2 != '{top_two[1]}'
        AND rank3 != '{top_two[1]}' AND rank4 != '{top_two[1]}')
        """
        cursor.execute(cmd)
        count1 = cursor.fetchone()[0]
        
        cmd = f"""
        SELECT COUNT(*) FROM votes
        WHERE (rank1 == '{top_two[1]}' AND rank2 == '{top_two[0]}')
        OR (rank1 == '{top_two[1]}' AND rank3 == '{top_two[0]}')
        OR (rank1 == '{top_two[1]}' AND rank4 == '{top_two[0]}')
        OR (rank1 == '{top_two[1]}' AND rank5 == '{top_two[0]}')
        OR (rank2 == '{top_two[1]}' AND rank3 == '{top_two[0]}')
        OR (rank2 == '{top_two[1]}' AND rank4 == '{top_two[0]}')
        OR (rank2 == '{top_two[1]}' AND rank5 == '{top_two[0]}')
        OR (rank3 == '{top_two[1]}' AND rank4 == '{top_two[0]}')
        OR (rank3 == '{top_two[1]}' AND rank5 == '{top_two[0]}')
        OR (rank4 == '{top_two[1]}' AND rank5 == '{top_two[0]}')
        OR (rank5 == '{top_two[1]}' AND rank1 != '{top_two[0]}' AND rank2 != '{top_two[0]}'
        AND rank3 != '{top_two[0]}' AND rank4 != '{top_two[0]}')
        """
        cursor.execute(cmd)
        count2 = cursor.fetchone()[0]
        
    return list(zip(top_two,[count1,count2]))

In [93]:
TopTwo()

[('Joseph R. Biden', 6480), ('Bernie Sanders', 5696)]